# Explore: Typing Module
This notebook contains code snippets to better understand type hints using the ``typing`` module. 

## Union of Generic and Concrete
Can a ``Union`` contain both a type variable and a concrete type?

In [1]:
import typing

In [2]:
import enum

In [3]:
class State(enum.Enum):
    UNCHECKED = enum.auto()
    UNDEFINED = enum.auto()
    CHECKED = enum.auto()

In [4]:
T = typing.TypeVar('T')

In [5]:
Value = typing.Union[State, T]

In [6]:
Value

typing.Union[__main__.State, ~T]

**Conclusion:** The preliminary answer is yes, a `Union` may contain generic and concrete types. In a separate check, ``Mypy`` did not object.

**Open**: use in non-trivial code.

## Inheritance from Generic Type
Built-in function `isinstance(object, class)` succeeds when `class` is generic but `isinstance(object, class[type])` raises a `TypeError`. This section demonstrates the behavior and looks at properties of classes derived from a generic class.

In [7]:
ValueAny = typing.TypeVar('ValueAny')
class Base(typing.Generic[ValueAny]):
    """Generic base class."""
    pass

Function ``isinstance`` can check an object against a generic class

In [8]:
item_int = Base[int]()
item_int
isinstance(item_int, Base)

True

Function ``isinstance`` raises `TypeError` after generic type is fixed (that is, after the generic is subscripted).

In [9]:
try:
    isinstance(item_int, Base[int])
except TypeError as err:
    print(err)

Subscripted generics cannot be used with class and instance checks


A wrapper class gives ``isinstance`` a concrete target.

In [10]:
class ChildInt(Base[int]):
    """Concrete child class."""
    pass

In [11]:
try:
    issubclass(ChildInt, Base[int])
except TypeError as err:
    print(err)

Subscripted generics cannot be used with class and instance checks


In [12]:
item_child_int = ChildInt()
isinstance(item_child_int, ChildInt)

True

A look at type properties provides insight into the behavior. First, type information for `item_child_int` includes both `ChildInt` and `Base[int]`.

In [13]:
print('type(item_child_int):          {}'.format(type(item_child_int)))
print('item_child_int.__class__:      {}'.format(item_child_int.__class__))
print('item_child_int.__orig_bases__: {}'.format(item_child_int.__orig_bases__))

type(item_child_int):          <class '__main__.ChildInt'>
item_child_int.__class__:      <class '__main__.ChildInt'>
item_child_int.__orig_bases__: (__main__.Base[int],)


In [14]:
base_int = Base[int]()
base_int

The type information for `ChildInt` includes both `Base[int]` and `Base` but in different ways (`__orig_bases__` and `__bases__`, respectively). The MRO of `ChildInt` does not include `Base[int]`.

In [15]:
print('type(ChildInt):          {}'.format(type(ChildInt)))
print('ChildInt.__class__:      {}'.format(ChildInt.__class__))
print('ChildInt.__orig_bases__: {}'.format(ChildInt.__orig_bases__))
print('ChildInt.__bases__:      {}'.format(ChildInt.__bases__))
print('ChildInt.__mro__:')
for c in ChildInt.__mro__:
    print('\t{}'.format(c))

type(ChildInt):          <class 'type'>
ChildInt.__class__:      <class 'type'>
ChildInt.__orig_bases__: (__main__.Base[int],)
ChildInt.__bases__:      (<class '__main__.Base'>,)
ChildInt.__mro__:
	<class '__main__.ChildInt'>
	<class '__main__.Base'>
	<class 'typing.Generic'>
	<class 'object'>


`Base[int]` is a `_GenericAlias` rather than a type. The `__args__` attribute shows that type `int` fixes the generic type.

(See also PEP 585 for changes in Python 3.9.)

In [16]:
print('type(Base[int]):           {}'.format(type(Base[int])))
print('Base[int].__class__:       {}'.format(Base[int].__class__))
print('Base[int].__args__:        {}'.format(Base[int].__args__))
print('Base[int].__origin__:      {}'.format(Base[int].__origin__))

type(Base[int]):           <class 'typing._GenericAlias'>
Base[int].__class__:       <class 'typing._GenericAlias'>
Base[int].__args__:        (<class 'int'>,)
Base[int].__origin__:      <class '__main__.Base'>


Object `base_int` directly derived from `Base[int]` refers to `Base` and its generic parbaseent.

In [17]:
print('type(base_int):          {}'.format(type(base_int)))
print('base_int.__class__:      {}'.format(base_int.__class__))
print('base_int.__orig_bases__: {}'.format(base_int.__orig_bases__))

type(base_int):          <class '__main__.Base'>
base_int.__class__:      <class '__main__.Base'>
base_int.__orig_bases__: (typing.Generic[~ValueAny],)


`Base` is a type with Generic base.

In [18]:
print('type(Base):          {}'.format(type(Base)))
print('Base.__class__:      {}'.format(Base.__class__))
print('Base.__orig_bases__: {}'.format(Base.__orig_bases__))
print('Base.__bases__:      {}'.format(Base.__bases__))
print('Base.__mro__:')
for c in Base.__mro__:
    print('\t{}'.format(c))
print('Base.__subclasses(): {}'.format(Base.__subclasses__()))

type(Base):          <class 'type'>
Base.__class__:      <class 'type'>
Base.__orig_bases__: (typing.Generic[~ValueAny],)
Base.__bases__:      (<class 'typing.Generic'>,)
Base.__mro__:
	<class '__main__.Base'>
	<class 'typing.Generic'>
	<class 'object'>
Base.__subclasses(): [<class '__main__.ChildInt'>]


**Conclusion:** A subscripted generic class is an alias not a class. A class derived from a subscripted generic class is a class